inspyred para rosenbrock funcional

In [ ]:
import random
import time
import csv
import os
import socket
import numpy as np
from codecarbon import EmissionsTracker
import inspyred
from inspyred import ec

# ------------------------------------------------------------
# Configuración del problema
# ------------------------------------------------------------
INDIVIDUAL_SIZE = 1024
MAX_GENERATIONS = 1000000
MAX_TIME_SECONDS = 120
RUNS_PER_CONFIG = 10

POPULATION_SIZES = [2**6, 2**10, 2**14]
CROSSOVER_RATES  = [0.01, 0.2, 0.8]

# -------------------- Tasas de mutación --------------------
MUTATION_RATE_INDIVIDUAL = 0.1   # p_ind  (se sortea 1 vez por individuo)
MUTATION_RATE_BIT        = 0.1   # p_bit  (se sortea por gen si el indiv. fue elegido)
# -------------------------------------------------------------

# --- Parámetros para Rosenbrock ----------------------------
LOWER_BOUND = -5.12
UPPER_BOUND = 5.12
# El mínimo global conocido de Rosenbrock es 0 cuando todos xi=1
KNOWN_OPTIMUM = 0.0
# Valor de referencia para el peor caso (estimación)
WORST_CASE_VALUE = 1e10
# Definir un valor fijo para la normalización
F_MAX = INDIVIDUAL_SIZE * 40000  
# -------------------------------------------------------------

COMPUTER_NAME   = socket.gethostname()
results_file    = f'resultados_rosenbrock_inspyred_{COMPUTER_NAME}.csv'
codecarbon_file = f'codecarbon_rosenbrock_inspyred_{COMPUTER_NAME}.csv'

# ------------------------------------------------------------
# Clases y funciones auxiliares
# ------------------------------------------------------------
class EvolutionStats:
    def __init__(self):
        self.initial_fitness   = 0
        self.best_fitness      = 0
        self.gen_best_fitness  = 0
        self.current_generation = 0
        self.termination_cause = "timeout"
        # Almacenar el mejor valor real de Rosenbrock 
        self.best_raw_value = float('inf')
        # Almacenar el peor valor visto para normalización dinámica
        self.worst_raw_value = 0

# Instancia global para almacenar estadísticas
stats = EvolutionStats()

# Función de fitness (Rosenbrock normalizado entre 0 y 1)
def rosenbrock_fitness(candidates, args):
    global stats
    fitness_values = []
    
    for candidate in candidates:
        # Calcular valor Rosenbrock (a minimizar)
        raw_value = 0
        for i in range(len(candidate) - 1):
            raw_value += 100 * (candidate[i+1] - candidate[i]**2)**2 + (candidate[i] - 1)**2
        
        # Limitar valores extremos para evitar problemas numéricos
        if raw_value > WORST_CASE_VALUE:
            raw_value = WORST_CASE_VALUE
            
        if raw_value > stats.worst_raw_value:
            stats.worst_raw_value = raw_value
            
        if raw_value < stats.best_raw_value:
            stats.best_raw_value = raw_value

        # Reemplazar la normalización dinámica
        normalized_fitness = 1.0 - (raw_value / F_MAX)
        
        # Asegurar que el fitness esté en el rango [0,1]
        normalized_fitness = max(0.0, min(1.0, normalized_fitness))
        
        fitness_values.append(normalized_fitness)
    
    return fitness_values

# Generador de individuos reales
def generator(random, args):
    size = args.get('individual_size', INDIVIDUAL_SIZE)
    lower_bound = args.get('lower_bound', LOWER_BOUND)
    upper_bound = args.get('upper_bound', UPPER_BOUND)
    
    # Inicialización uniforme
    return [random.uniform(lower_bound, upper_bound) for _ in range(size)]

# ------------------------------------------------------------
# Variador personalizado
# ------------------------------------------------------------
def real_mutation(random, candidates, args):

    p_ind = args.get('mutation_individual_rate', MUTATION_RATE_INDIVIDUAL)
    p_bit = args.get('mutation_bit_rate', MUTATION_RATE_BIT)
    lower_bound = args.get('lower_bound', LOWER_BOUND)
    upper_bound = args.get('upper_bound', UPPER_BOUND)
    
    # Factor de escala para la mutación gaussiana
    sigma = (upper_bound - lower_bound) * 0.1
    
    mutated = []
    for cand in candidates:
        if random.random() < p_ind:
            new_cand = cand[:]
            for i in range(len(new_cand)):
                if random.random() < p_bit:
                    # Mutación gaussiana
                    delta = random.gauss(0, sigma)
                    new_cand[i] += delta
                    # Asegurar que se mantiene dentro de límites
                    new_cand[i] = max(min(new_cand[i], upper_bound), lower_bound)
            mutated.append(new_cand)
        else:
            mutated.append(cand)
    return mutated

# ------------------------------------------------------------
# Implementación personalizada de SBX para evitar desbordamiento
# ------------------------------------------------------------
def safe_sbx_crossover(random, candidates, args):
    
    crossover_rate = args.setdefault('crossover_rate', 1.0)
    children = []
    
    di = args.setdefault('sbx_distribution_index', 2.0)
    
    for i, (p1, p2) in enumerate(zip(candidates[::2], candidates[1::2])):
        if random.random() < crossover_rate:
            c1 = p1[:]
            c2 = p2[:]
            
            for j in range(len(c1)):
                # Evitar la división por cero
                if abs(p1[j] - p2[j]) > 1e-10:
                    if p1[j] < p2[j]:
                        y1, y2 = p1[j], p2[j]
                    else:
                        y1, y2 = p2[j], p1[j]
                        
                    # Cálculos SBX con protección contra desbordamiento
                    try:
                        # Limitar la magnitud del exponente para evitar desbordamiento
                        beta = 1.0 + (2.0 * (y1 - LOWER_BOUND) / (y2 - y1))
                        alpha = 2.0 - min(100.0, beta**(di + 1.0))
                        
                        u = random.random()
                        if u <= 1.0 / alpha:
                            beta_q = (u * alpha) ** (1.0 / (di + 1.0))
                        else:
                            beta_q = (1.0 / (2.0 - u * alpha)) ** (1.0 / (di + 1.0))
                        
                        c1[j] = 0.5 * ((y1 + y2) - beta_q * (y2 - y1))
                        c2[j] = 0.5 * ((y1 + y2) + beta_q * (y2 - y1))
                        
                        # Mantener dentro de límites
                        c1[j] = max(LOWER_BOUND, min(UPPER_BOUND, c1[j]))
                        c2[j] = max(LOWER_BOUND, min(UPPER_BOUND, c2[j]))
                        
                        # Intercambiar si el orden se invirtió
                        if p1[j] > p2[j]:
                            c1[j], c2[j] = c2[j], c1[j]
                            
                    except (OverflowError, ValueError, ZeroDivisionError):
                        # En caso de error, aplicar cruce aritmético simple
                        blend = random.random()
                        c1[j] = blend * p1[j] + (1 - blend) * p2[j]
                        c2[j] = (1 - blend) * p1[j] + blend * p2[j]
            
            children.append(c1)
            children.append(c2)
        else:
            children.append(p1)
            children.append(p2)
    return children

# ------------------------------------------------------------
# Terminador combinado tiempo y generaciones
# ------------------------------------------------------------
def time_generation_termination(population, num_generations, num_evaluations, args):
    global stats
    max_time        = args.get('max_time', MAX_TIME_SECONDS)
    max_generations = args.get('max_generations', MAX_GENERATIONS)
    start_time      = args.get('start_time')

    # Actualizar generación actual
    stats.current_generation = num_generations

    if num_generations >= max_generations:
        stats.termination_cause = "max_generations"
        return True
    if time.time() - start_time >= max_time:
        stats.termination_cause = "timeout"
        return True
    return False

# ------------------------------------------------------------
# Observador para acumular estadísticas básicas
# ------------------------------------------------------------
def stats_observer(population, num_generations, num_evaluations, args):
    global stats
    if not population:
        return
    
    current_best = max(ind.fitness for ind in population)
    
    if num_generations == 0:
        stats.initial_fitness = current_best
    if current_best > stats.best_fitness:
        stats.best_fitness     = current_best
        stats.gen_best_fitness = num_generations

# ------------------------------------------------------------
# Ejecución principal
# ------------------------------------------------------------
def main():
    # Crear archivo CSV de resultados si no existe
    file_exists = os.path.isfile(results_file)
    with open(results_file, 'a', newline='') as csvfile:
        fieldnames = ['population_size', 'crossover_rate', 'mutation_individual_rate',
                      'mutation_bit_rate', 'run', 'generations', 'initial_fitness',
                      'best_fitness', 'fitness_variation', 'time', 'energy_consumed',
                      'rosenbrock_value', 'worst_rosenbrock_seen']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()

        # Bucle sobre combinaciones de parámetros
        for pop_size in POPULATION_SIZES:
            for cx_rate in CROSSOVER_RATES:
                for run in range(1, RUNS_PER_CONFIG + 1):
                    print(f"Ejecutando: Población={pop_size}, Cruce={cx_rate}, "
                          f"Run={run}")

                    # Reiniciar estadísticas
                    global stats
                    stats = EvolutionStats()

                    # Semilla aleatoria única
                    rand = random.Random()
                    rand.seed(time.time() + run)

                    # Configurar GA
                    ea = ec.GA(rand)
                    ea.selector = inspyred.ec.selectors.tournament_selection
                    
                    ea.variator = [
                        safe_sbx_crossover,
                        real_mutation
                    ]
                    ea.replacer   = inspyred.ec.replacers.generational_replacement
                    ea.terminator = time_generation_termination
                    ea.observer   = stats_observer

                    tracker_id = f"rosenbrock_p{pop_size}_cx{cx_rate}_r{run}"
                    output_dir = os.path.dirname(os.path.abspath(codecarbon_file)) or '.'
                    output_file = os.path.basename(codecarbon_file)

                    # Asegurarnos de que el directorio existe
                    if not os.path.exists(output_dir):
                        try:
                            os.makedirs(output_dir)
                            print(f"Creado directorio: {output_dir}")
                        except Exception as e:
                            print(f"Error al crear directorio: {e}")

                    # Configurar variables de entorno
                    os.environ["CODECARBON_RUN_ID"] = tracker_id

                    try:
                        tracker = EmissionsTracker(
                            project_name="Rosenbrock_Experiment",
                            output_dir=output_dir,
                            output_file=output_file,
                            save_to_file=True,
                            log_level="critical",  # Cambiar a 'info' para depuración
                            tracking_mode="process",
                        )
                        print(f"Tracker inicializado: ID={tracker_id}")
                        
                        tracker.start()
                        start_time = time.time()
                        tracker_started = True
                        print(f"Emisiones: tracking iniciado")
                    except Exception as e:
                        print(f"Error con CodeCarbon: {e}")
                        tracker_started = False
                        start_time = time.time()

                    # Ejecutar evolución
                    try:
                        final_pop = ea.evolve(
                            generator      = generator,
                            evaluator      = rosenbrock_fitness,
                            pop_size       = pop_size,
                            maximize       = True,  # Maximizamos la versión normalizada
                            bounder        = inspyred.ec.Bounder(LOWER_BOUND, UPPER_BOUND),
                            max_generations= MAX_GENERATIONS,
                            start_time     = start_time,
                            max_time       = MAX_TIME_SECONDS,
                            num_selected   = pop_size,
                            individual_size= INDIVIDUAL_SIZE,
                            crossover_rate = cx_rate,
                            # → Parámetros para SBX y mutación
                            mutation_individual_rate = MUTATION_RATE_INDIVIDUAL,
                            mutation_bit_rate        = MUTATION_RATE_BIT,
                            tournament_size          = 2,
                            # → Límites para el generador y mutador
                            lower_bound    = LOWER_BOUND,
                            upper_bound    = UPPER_BOUND,
                            # → Parámetro específico para SBX (reducido para evitar errores)
                            sbx_distribution_index = 2.0
                        )
                    except Exception as e:
                        print(f"Error durante la evolución: {e}")
                        # Si hay error, intentar continuar con la información que tenemos
                        final_pop = []

                    # Tiempo y emisiones
                    elapsed_time = time.time() - start_time
                    
                    # Guardar emisiones
                    if tracker_started:
                        try:
                            emissions = tracker.stop()
                            print(f"Emisiones registradas: {emissions or 0:.6f} kWh")
                        except Exception as e:
                            print(f"Error al detener tracker: {e}")
                            emissions = 0
                    else:
                        emissions = 0

                    # Verificar si el archivo de emisiones existe después de la ejecución
                    emissions_file_path = os.path.join(output_dir, output_file)
                    if os.path.exists(emissions_file_path):
                        print(f"Archivo de emisiones creado correctamente")
                    else:
                        print(f"ADVERTENCIA: No se creó el archivo de emisiones")
                        # Intentar guardar manualmente las emisiones
                        try:
                            with open(emissions_file_path, 'a', newline='') as f:
                                f.write(f"manual_entry,{tracker_id},{emissions or 0},{time.time()}\n")
                            print(f"Creado manualmente un registro básico de emisiones")
                        except Exception as e:
                            print(f"Error al crear registro manual: {e}")

                    # Estadísticas finales
                    generations       = stats.current_generation + 1
                    initial_fitness   = stats.initial_fitness
                    best_fitness      = stats.best_fitness
                    fitness_variation = best_fitness - initial_fitness

                    # Registro en CSV
                    writer.writerow({
                        'population_size'        : pop_size,
                        'crossover_rate'         : cx_rate,
                        'mutation_individual_rate': MUTATION_RATE_INDIVIDUAL,
                        'mutation_bit_rate'      : MUTATION_RATE_BIT,
                        'run'                    : run,
                        'generations'            : generations,
                        'initial_fitness'        : initial_fitness,
                        'best_fitness'           : best_fitness,
                        'fitness_variation'      : fitness_variation,
                        'time'                   : elapsed_time,
                        'energy_consumed'        : emissions,
                        'rosenbrock_value'       : stats.best_raw_value,
                        'worst_rosenbrock_seen'  : stats.worst_raw_value
                    })
                    csvfile.flush()

                    print(f"Generaciones: {generations} | "
                          f"Aptitud inicial={initial_fitness:.6f} → "
                          f"mejor={best_fitness:.6f} | "
                          f"Δ={fitness_variation:.6f}")
                    print(f"Valor Rosenbrock: mejor={stats.best_raw_value:.4e}, "
                          f"peor visto={stats.worst_raw_value:.4e}")
                    print(f"Tiempo: {elapsed_time:.2f}s | "
                          f"Energía: {emissions:.4f} kWh | "
                          f"Parada: {stats.termination_cause}")
                    print("-" * 60)

if __name__ == "__main__":
    main()

inspyred para schwefel funcional

In [ ]:
import random
import time
import csv
import os
import socket
import numpy as np
from codecarbon import EmissionsTracker
import inspyred
from inspyred import ec

# ------------------------------------------------------------
# Configuración del problema
# ------------------------------------------------------------
INDIVIDUAL_SIZE = 1024
MAX_GENERATIONS = 1000000
MAX_TIME_SECONDS = 120
RUNS_PER_CONFIG = 10

POPULATION_SIZES = [2**6, 2**10, 2**14]
CROSSOVER_RATES  = [0.01, 0.2, 0.8]

# ----------------- Tasas de mutación ------------------------
MUTATION_RATE_INDIVIDUAL = 0.1   # p_ind  (se sortea 1 vez por individuo)
MUTATION_RATE_BIT        = 0.1   # p_bit  (se sortea por gen si el indiv. fue elegido)
# -------------------------------------------------------------

# --- Parámetros para Schwefel ----------------------------
LOWER_BOUND = -500  # Límite típico para Schwefel
UPPER_BOUND = 500   # Límite típico para Schwefel
# El mínimo global conocido de Schwefel es en 420.9687 para cada xi
KNOWN_OPTIMUM = 0.0
# Valor de referencia para el peor caso (estimación)
WORST_CASE_VALUE = INDIVIDUAL_SIZE * (418.9829 + UPPER_BOUND)
# -------------------------------------------------------------

COMPUTER_NAME   = socket.gethostname()
results_file    = f'resultados_schwefel_inspyred_{COMPUTER_NAME}.csv'
codecarbon_file = f'codecarbon_schwefel_inspyred_{COMPUTER_NAME}.csv'

# ------------------------------------------------------------
# Clases y funciones auxiliares
# ------------------------------------------------------------
class EvolutionStats:
    def __init__(self):
        self.initial_fitness   = 0
        self.best_fitness      = 0
        self.gen_best_fitness  = 0
        self.current_generation = 0
        self.termination_cause = "timeout"
        # Almacenar el mejor valor real para conversión
        self.best_raw_value = float('inf')
        # Almacenar el peor valor visto para normalización dinámica
        self.worst_raw_value = 0

# Instancia global para almacenar estadísticas
stats = EvolutionStats()

# Función de fitness (Schwefel normalizado entre 0 y 1)
def schwefel_fitness(candidates, args):
    global stats
    fitness_values = []
    
    for candidate in candidates:
        # Calcular valor Schwefel (a minimizar)
        raw_value = 0
        D = len(candidate)
        
        for i in range(D):
            raw_value += -candidate[i] * np.sin(np.sqrt(abs(candidate[i])))
        
        raw_value = 418.9829 * D - raw_value
        
        if raw_value > WORST_CASE_VALUE:
            raw_value = WORST_CASE_VALUE
            
        if raw_value > stats.worst_raw_value:
            stats.worst_raw_value = raw_value
            
        if raw_value < stats.best_raw_value:
            stats.best_raw_value = raw_value
        
        # Estimación del valor máximo posible (peor caso teórico)
        theoretical_worst = 418.9829 * D + D * UPPER_BOUND
        
        # Usar el peor valor visto hasta ahora o el teórico, el que sea mayor
        normalization_factor = max(stats.worst_raw_value, theoretical_worst)
        
        # Normalización invertida usando el factor combinado
        normalized_fitness = 1.0 - (raw_value / normalization_factor)
        
        normalized_fitness = max(0.0, min(1.0, normalized_fitness))
        fitness_values.append(normalized_fitness)
    
    return fitness_values

# Generador de individuos reales
def generator(random, args):
    size = args.get('individual_size', INDIVIDUAL_SIZE)
    lower_bound = args.get('lower_bound', LOWER_BOUND)
    upper_bound = args.get('upper_bound', UPPER_BOUND)
    

    # distribuir uniformemente por todo el espacio de búsqueda
    return [random.uniform(lower_bound, upper_bound) for _ in range(size)]

# ------------------------------------------------------------
# Variador personalizado
# ------------------------------------------------------------
def real_mutation(random, candidates, args):
    """
    Versión adaptada del mutador para valores reales:
    1) p_ind = probabilidad de mutar cada individuo.
    2) Si se muta, p_bit = probabilidad de mutar cada gen.
    3) Para cada gen seleccionado, se aplica una perturbación gaussiana.
    """
    p_ind = args.get('mutation_individual_rate', MUTATION_RATE_INDIVIDUAL)
    p_bit = args.get('mutation_bit_rate', MUTATION_RATE_BIT)
    lower_bound = args.get('lower_bound', LOWER_BOUND)
    upper_bound = args.get('upper_bound', UPPER_BOUND)
    
    # Factor de escala para la mutación gaussiana
    sigma = (upper_bound - lower_bound) * 0.1
    
    mutated = []
    for cand in candidates:
        # Paso 1: ¿se muta el individuo?
        if random.random() < p_ind:
            new_cand = cand[:]
            # Paso 2: para cada gen, decide si mutar
            for i in range(len(new_cand)):
                if random.random() < p_bit:
                    # Mutación gaussiana
                    delta = random.gauss(0, sigma)
                    new_cand[i] += delta
                    # Asegurar que se mantiene dentro de límites
                    new_cand[i] = max(min(new_cand[i], upper_bound), lower_bound)
            mutated.append(new_cand)
        else:
            mutated.append(cand)
    return mutated

# ------------------------------------------------------------
# Implementación personalizada de SBX para evitar desbordamiento
# ------------------------------------------------------------
def safe_sbx_crossover(random, candidates, args):
    """
    Implementación personalizada de SBX (Simulated Binary Crossover)
    con protección contra desbordamiento numérico.
    """
    crossover_rate = args.setdefault('crossover_rate', 1.0)
    children = []
    
    # Un valor más bajo para el índice de distribución evita el desbordamiento
    di = args.setdefault('sbx_distribution_index', 2.0)  
    
    for i, (p1, p2) in enumerate(zip(candidates[::2], candidates[1::2])):
        if random.random() < crossover_rate:
            c1 = p1[:]
            c2 = p2[:]
            
            for j in range(len(c1)):
                # Evitar la división por cero
                if abs(p1[j] - p2[j]) > 1e-10:
                    if p1[j] < p2[j]:
                        y1, y2 = p1[j], p2[j]
                    else:
                        y1, y2 = p2[j], p1[j]
                        
                    # Cálculos SBX con protección contra desbordamiento
                    try:
                        # Limitar la magnitud del exponente para evitar desbordamiento
                        beta = 1.0 + (2.0 * (y1 - LOWER_BOUND) / (y2 - y1))
                        alpha = 2.0 - min(100.0, beta**(di + 1.0))
                        
                        u = random.random()
                        if u <= 1.0 / alpha:
                            beta_q = (u * alpha) ** (1.0 / (di + 1.0))
                        else:
                            beta_q = (1.0 / (2.0 - u * alpha)) ** (1.0 / (di + 1.0))
                        
                        c1[j] = 0.5 * ((y1 + y2) - beta_q * (y2 - y1))
                        c2[j] = 0.5 * ((y1 + y2) + beta_q * (y2 - y1))
                        
                        # Mantener dentro de límites
                        c1[j] = max(LOWER_BOUND, min(UPPER_BOUND, c1[j]))
                        c2[j] = max(LOWER_BOUND, min(UPPER_BOUND, c2[j]))
                        
                        # Intercambiar si el orden se invirtió
                        if p1[j] > p2[j]:
                            c1[j], c2[j] = c2[j], c1[j]
                            
                    except (OverflowError, ValueError, ZeroDivisionError):
                        # En caso de error, aplicar cruce aritmético simple
                        blend = random.random()
                        c1[j] = blend * p1[j] + (1 - blend) * p2[j]
                        c2[j] = (1 - blend) * p1[j] + blend * p2[j]
            
            children.append(c1)
            children.append(c2)
        else:
            children.append(p1)
            children.append(p2)
    return children

# ------------------------------------------------------------
# Terminador combinado tiempo + generaciones
# ------------------------------------------------------------
def time_generation_termination(population, num_generations, num_evaluations, args):
    global stats
    max_time        = args.get('max_time', MAX_TIME_SECONDS)
    max_generations = args.get('max_generations', MAX_GENERATIONS)
    start_time      = args.get('start_time')

    # Actualizar generación actual
    stats.current_generation = num_generations

    if num_generations >= max_generations:
        stats.termination_cause = "max_generations"
        return True
    if time.time() - start_time >= max_time:
        stats.termination_cause = "timeout"
        return True
    return False

# ------------------------------------------------------------
# Observador para acumular estadísticas básicas
# ------------------------------------------------------------
def stats_observer(population, num_generations, num_evaluations, args):
    global stats
    if not population:
        return
    
    current_best = max(ind.fitness for ind in population)
    
    # Para primeras generaciones o cada 50, mostrar información
    if num_generations < 3 or num_generations % 50 == 0:
        print(f"  Gen {num_generations}: fitness={current_best:.6f}, "
              f"raw={stats.best_raw_value:.2e}, "
              f"worst_seen={stats.worst_raw_value:.2e}")
    
    if num_generations == 0:
        stats.initial_fitness = current_best
    if current_best > stats.best_fitness:
        stats.best_fitness     = current_best
        stats.gen_best_fitness = num_generations

# ------------------------------------------------------------
# Ejecución principal
# ------------------------------------------------------------
def main():
    # Crear archivo CSV de resultados si no existe
    file_exists = os.path.isfile(results_file)
    with open(results_file, 'a', newline='') as csvfile:
        fieldnames = ['population_size', 'crossover_rate', 'mutation_individual_rate',
                      'mutation_bit_rate', 'run', 'generations', 'initial_fitness',
                      'best_fitness', 'fitness_variation', 'time', 'energy_consumed',
                      'schwefel_value', 'worst_schwefel_seen']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()

        # Bucle sobre combinaciones de parámetros
        for pop_size in POPULATION_SIZES:
            for cx_rate in CROSSOVER_RATES:
                for run in range(1, RUNS_PER_CONFIG + 1):
                    print(f"Ejecutando: Población={pop_size}, Cruce={cx_rate}, "
                          f"Run={run}")

                    # Reiniciar estadísticas
                    global stats
                    stats = EvolutionStats()

                    # Semilla aleatoria única
                    rand = random.Random()
                    rand.seed(time.time() + run)

                    # Configurar GA
                    ea = ec.GA(rand)
                    ea.selector = inspyred.ec.selectors.tournament_selection
                    
                    # Usar nuestra implementación SBX segura en lugar de la original
                    ea.variator = [
                        safe_sbx_crossover,
                        real_mutation
                    ]
                    ea.replacer   = inspyred.ec.replacers.generational_replacement
                    ea.terminator = time_generation_termination
                    ea.observer   = stats_observer

                    # CodeCarbon - Configuración mejorada
                    tracker_id = f"schwefel_p{pop_size}_cx{cx_rate}_r{run}"
                    output_dir = os.path.dirname(os.path.abspath(codecarbon_file)) or '.'
                    output_file = os.path.basename(codecarbon_file)

                    # Asegurarnos de que el directorio existe
                    if not os.path.exists(output_dir):
                        try:
                            os.makedirs(output_dir)
                            print(f"Creado directorio: {output_dir}")
                        except Exception as e:
                            print(f"Error al crear directorio: {e}")

                    # Configurar variables de entorno
                    os.environ["CODECARBON_RUN_ID"] = tracker_id

                    try:
                        tracker = EmissionsTracker(
                            project_name="Schwefel_Experiment",
                            output_dir=output_dir,
                            output_file=output_file,
                            save_to_file=True,
                            log_level="critical",
                            tracking_mode="process",
                        )
                        print(f"Tracker inicializado: ID={tracker_id}")
                        
                        tracker.start()
                        start_time = time.time()
                        tracker_started = True
                        print(f"Emisiones: tracking iniciado")
                    except Exception as e:
                        print(f"Error con CodeCarbon: {e}")
                        tracker_started = False
                        start_time = time.time()

                    # Ejecutar evolución
                    try:
                        final_pop = ea.evolve(
                            generator      = generator,
                            evaluator      = schwefel_fitness,  # Cambiado a Schwefel
                            pop_size       = pop_size,
                            maximize       = True,  # Maximizamos la versión normalizada
                            bounder        = inspyred.ec.Bounder(LOWER_BOUND, UPPER_BOUND),
                            max_generations= MAX_GENERATIONS,
                            start_time     = start_time,
                            max_time       = MAX_TIME_SECONDS,
                            num_selected   = pop_size,
                            individual_size= INDIVIDUAL_SIZE,
                            crossover_rate = cx_rate,
                            # → Parámetros para SBX y mutación
                            mutation_individual_rate = MUTATION_RATE_INDIVIDUAL,
                            mutation_bit_rate        = MUTATION_RATE_BIT,
                            tournament_size          = 2,
                            # → Límites para el generador y mutador
                            lower_bound    = LOWER_BOUND,
                            upper_bound    = UPPER_BOUND,
                            # → Parámetro específico para SBX (reducido para evitar errores)
                            sbx_distribution_index = 2.0
                        )
                    except Exception as e:
                        print(f"Error durante la evolución: {e}")
                        # Si hay error, intentar continuar con la información que tenemos
                        final_pop = []

                    # Tiempo y emisiones
                    elapsed_time = time.time() - start_time
                    
                    # Guardar emisiones
                    if tracker_started:
                        try:
                            emissions = tracker.stop()
                            print(f"Emisiones registradas: {emissions or 0:.6f} kWh")
                        except Exception as e:
                            print(f"Error al detener tracker: {e}")
                            emissions = 0
                    else:
                        emissions = 0

                    # Verificar si el archivo de emisiones existe después de la ejecución
                    emissions_file_path = os.path.join(output_dir, output_file)
                    if os.path.exists(emissions_file_path):
                        print(f"Archivo de emisiones creado correctamente")
                    else:
                        print(f"ADVERTENCIA: No se creó el archivo de emisiones")
                        # Intentar guardar manualmente las emisiones
                        try:
                            with open(emissions_file_path, 'a', newline='') as f:
                                f.write(f"manual_entry,{tracker_id},{emissions or 0},{time.time()}\n")
                            print(f"Creado manualmente un registro básico de emisiones")
                        except Exception as e:
                            print(f"Error al crear registro manual: {e}")

                    # Estadísticas finales
                    generations       = stats.current_generation + 1
                    initial_fitness   = stats.initial_fitness
                    best_fitness      = stats.best_fitness
                    fitness_variation = best_fitness - initial_fitness

                    # Registro en CSV
                    writer.writerow({
                        'population_size'        : pop_size,
                        'crossover_rate'         : cx_rate,
                        'mutation_individual_rate': MUTATION_RATE_INDIVIDUAL,
                        'mutation_bit_rate'      : MUTATION_RATE_BIT,
                        'run'                    : run,
                        'generations'            : generations,
                        'initial_fitness'        : initial_fitness,
                        'best_fitness'           : best_fitness,
                        'fitness_variation'      : fitness_variation,
                        'time'                   : elapsed_time,
                        'energy_consumed'        : emissions,
                        'schwefel_value'         : stats.best_raw_value,
                        'worst_schwefel_seen'    : stats.worst_raw_value
                    })
                    csvfile.flush()

                    print(f"Generaciones: {generations} | "
                          f"Aptitud inicial={initial_fitness:.6f} → "
                          f"mejor={best_fitness:.6f} | "
                          f"Δ={fitness_variation:.6f}")
                    print(f"Valor Schwefel: mejor={stats.best_raw_value:.4e}, "
                          f"peor visto={stats.worst_raw_value:.4e}")
                    print(f"Tiempo: {elapsed_time:.2f}s | "
                          f"Energía: {emissions:.4f} kWh | "
                          f"Parada: {stats.termination_cause}")
                    print("-" * 60)

if __name__ == "__main__":
    main()